In [4]:
# 데이터 로드 

import pandas as pd

bank_df = pd.read_csv('UniversalBank.csv')
bank_df.head()


,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,PersonalLoan,SecuritiesAccount,CDAccount,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [5]:
# 학습에 사용할 특성변수 선택 

X = bank_df.drop(['ID','ZIPCode','PersonalLoan'],axis=1)
y = bank_df['PersonalLoan']


In [6]:
# 데이터 분할 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state = 1,stratify=y)

In [7]:
# 학습에 사용할 모델 개별 정의 

from sklearn.tree import DecisionTreeClassifier
# 결정트리 
from sklearn.neighbors import KNeighborsClassifier
# K 최근접이웃 
from sklearn.linear_model import LogisticRegression
# 로지스틱 회귀 모델 

logistic = LogisticRegression(solver='liblinear',
                              penalty='12',
                              C = 0.001,
                              random_state = 1)

tree = DecisionTreeClassifier(max_depth=None,
                              criterion = 'entropy',
                              random_state = 1)

knn = KNeighborsClassifier(n_neighbors=1,
                           p=2,
                           metric = 'minkowski')


In [12]:
# 학습에 사용할 모델 앙상블 정의 

from sklearn.ensemble import VotingClassifier
# 과반수 투표 (majority voting)

voting_estimators = [('logistic',logistic),('tree',tree),('knn',knn)]
voting = VotingClassifier(estimators = voting_estimators,
                          voting = 'soft')


In [13]:
# k-fold 교차검증을 통한 모델평가 

from sklearn.model_selection import cross_val_score # 교차타당도 # 추가

clf_labels = ['Logistic regression', 'Decision tree', 'KNN', 'Majority voting']
all_clf = [logistic, tree, knn, voting]

for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print("ROC AUC: %0.3f (+/- %0.3f) [%s]", (scores.mean(), scores.std(), label))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Logistic Regression supports only penalties in ['l1', 'l2', 'elasticnet', 'none'], got 12.

  FitFailedWarning)


ROC AUC: %0.3f (+/- %0.3f) [%s] (nan, nan, 'Logistic regression')
ROC AUC: %0.3f (+/- %0.3f) [%s] (0.9499227861055811, 0.032735680131811405, 'Decision tree')
ROC AUC: %0.3f (+/- %0.3f) [%s] (0.7120816883018249, 0.04722587272864442, 'KNN')
ROC AUC: %0.3f (+/- %0.3f) [%s] (nan, nan, 'Majority voting')


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Logistic Regression supports only penalties in ['l1', 'l2', 'elasticnet', 'none'], got 12.

  FitFailedWarning)


In [14]:
# grid search 방식을 이용한 모델 최적화 

from sklearn.model_selection import GridSearchCV
# 하이퍼파라미터 튜닝 
params = {'logistic__C': [0.001, 0.1, 100.0],
          'tree__max_depth': [1, 3, 5],
          'knn__n_neighbors': [1, 3, 5]}

grid = GridSearchCV(estimator=voting,
                    param_grid=params,
                    cv=10,
                    scoring='roc_auc',
                    iid=False)
grid.fit(X_train, y_train)

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.3f %r"
          % (grid.cv_results_['mean_test_score'][r], 
             grid.cv_results_['std_test_score'][r] / 2.0, 
             grid.cv_results_['params'][r]))
    
print('최적의 파타미터: %s' % grid.best_params_)
print('ACU: %.3f' % grid.best_score_)    



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Logistic Regression supports only penalties in ['l1', 'l2', 'elasticnet', 'none'], got 12.

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


ValueError: ignored